In [1]:
import pixiedust
pixiedust.enableJobMonitor()

Pixiedust database opened successfully


Succesfully enabled Spark Job Progress Monitor


In [1]:
"""
Extract head and content from file
"""
rdd = sc.textFile('../Project/Data/Broilers_Data.txt')

lineSpliter = rdd.map(lambda x: x.split('\t'))
print("The file has {0} lines\n".format(lineSpliter.count()))

oldHeader = ['MainKey']

header = ['Main_Key', 'Date', 'Customer_Code', 'House', 'Flock', 'Gene_Line', 'Birds_Begin', 'Hatch_Date', 'Arrive_Date',\
 'Remove_date', 'Deactivate_Date', 'Veterinarian', 'Hatchery', 'Age', 'Birds_Present', 'Birds_thinned', 'Death',\
 'Death_Cum', 'Total_Death_Rate', 'Alive_Rate', 'Body_Weight_g', 'Uniformity_Rate', 'Daily_Gaing', 'Avg_Daily_Gaing_Per_Day',\
 'Feed_Intake_Per_house_kg', 'FCR_Cum', 'Wheat_Per_Bird_Cum', 'Wheat_Per_Bird', 'Wheat_Per_Day',\
 'Feed_Intake_Per_Bird_Housed_Cum_kg', 'Feed_Intake_Per_Bird_g', 'Wheat_g', 'FCR', 'Water_l', 'Water_Intake_Per_Bird_ml',\
 'Water_Intake_Per_Bird_Cum_l', 'Water_Feed', 'Water_FeedCum', 'Comment']

# Codes in for-loop is useless now
for col in lineSpliter.take(1)[0]:
    oldHeader.append(col.strip()
                     .replace(' ', '')
                     .replace('#', '')
                     .replace('(', '')
                     .replace(')', '')
                     .replace('.', '')
                     .replace('/', 'Per')
                     .replace('%', 'Percentage'))

print("The header of this file is \n{0}\n".format(header))
print("The header of this file has {0} lines".format(len(header)))

data = lineSpliter.filter(lambda x: x[0] != "Date")
#print("Data in this file has {0} lines".format(data.count()))

def lengthInspector(rdd):
    lenCounter = rdd.map(lambda x: (len(x), 1))\
                 .reduceByKey(lambda x, y: x + y)
    lenList = lenCounter.collect()
    return lenList

lenList = lengthInspector(data)
print("Distribution of length of lines in this file: ")
print(lenList)

"""
Clean the empty lines, calculate main key and clean illegal main key
"""
def buildMainKey(line):
    mainKey = "Farm " + line[1].strip() + " House " + line[2].strip() + " Flock " + line[3].strip()
    newline = [mainKey] + line
    return newline

emptyClener = data.filter(lambda x: len(x) >= 38)
#print("Not empty data has {0} lines".format(emptyClener.count()))

mainKeyCounter = emptyClener.map(lambda x: buildMainKey(x))\
                            .map(lambda x: (x[0], 1))\
                            .reduceByKey(lambda x, y: x + y)
print("\nThere are {0} different main keys".format(mainKeyCounter.count()))

errorMainKeyCleaner = emptyClener.filter(lambda x: False if x[3] == "" else True)
print("There are {0} lines with vaild main key".format(errorMainKeyCleaner.count()))

cleanData = errorMainKeyCleaner.map(lambda x: buildMainKey(x))
cleanMainKeyCounter = cleanData.map(lambda x: (x[0], 1))\
                               .reduceByKey(lambda x, y: x + y)
mainKeysWithCounts = cleanMainKeyCounter.collect()
mainKeys = cleanMainKeyCounter.map(lambda x: x[0]).collect()
#print("The main keys are:")
for mainkey in mainKeys:
    pass#print(mainkey)

The file has 3678 lines

The header of this file is 
['Main_Key', 'Date', 'Customer_Code', 'House', 'Flock', 'Gene_Line', 'Birds_Begin', 'Hatch_Date', 'Arrive_Date', 'Remove_date', 'Deactivate_Date', 'Veterinarian', 'Hatchery', 'Age', 'Birds_Present', 'Birds_thinned', 'Death', 'Death_Cum', 'Total_Death_Rate', 'Alive_Rate', 'Body_Weight_g', 'Uniformity_Rate', 'Daily_Gaing', 'Avg_Daily_Gaing_Per_Day', 'Feed_Intake_Per_house_kg', 'FCR_Cum', 'Wheat_Per_Bird_Cum', 'Wheat_Per_Bird', 'Wheat_Per_Day', 'Feed_Intake_Per_Bird_Housed_Cum_kg', 'Feed_Intake_Per_Bird_g', 'Wheat_g', 'FCR', 'Water_l', 'Water_Intake_Per_Bird_ml', 'Water_Intake_Per_Bird_Cum_l', 'Water_Feed', 'Water_FeedCum', 'Comment']

The header of this file has 39 lines
Distribution of length of lines in this file: 
[(38, 3676), (1, 1)]

There are 76 different main keys
There are 3673 lines with vaild main key


In [2]:
"""

"""
mainKeyArror = range(1, 74)
numMainKey = dict(zip(mainKeyArror, mainKeys))
print(numMainKey)

dataIndexArror = range(39)
numDataIndex = dict(zip(dataIndexArror, header))
print(numDataIndex)

def dataTaker(mainKey, dataIndex):
    """
    Return a list of data under given main key and data idx
    """
    if type(mainKey) is int:
        mainKey = numMainKey[mainKey]
    if type(dataIndex) is str:
        for key in numDataIndex:
            if dataIndex == numDataIndex[key]: 
                dataIndex = key
                break
    
    result = cleanData.filter(lambda line: line[0] == mainKey)\
                      .map(lambda line: line[dataIndex])\
                      .collect() # dataIndex should not minus 1 because main key is line[0]
    return result

{1: 'Farm A House 1 Flock 2016-07', 2: 'Farm A House 1 Flock 2016-10', 3: 'Farm B House 3 Flock 2016-11', 4: 'Farm B House 1 Flock 2016-11', 5: 'Farm B House 3 Flock 2016-12', 6: 'Farm B House 2 Flock 2017-01', 7: 'Farm B House 2 Flock 17-3', 8: 'Farm B House 1 Flock 17-3', 9: 'Farm C House 2 Flock Koppel 11 stal 2', 10: 'Farm D House 1 Flock 2016-07', 11: 'Farm D House 3 Flock 2016-07', 12: 'Farm D House 2 Flock 2016-09', 13: 'Farm D House 2 Flock 2016-11', 14: 'Farm D House 3 Flock 2016-12', 15: 'Farm D House 1 Flock 2016-12', 16: 'Farm D House 2 Flock 2017-02', 17: 'Farm D House 3 Flock 2017-04', 18: 'Farm D House 2 Flock 2017-06', 19: 'Farm D House 3 Flock 2017-06', 20: 'Farm D House 1 Flock 2017-10', 21: 'Farm D House 3 Flock 2017-10', 22: 'Farm E House 5 Flock 2016-09', 23: 'Farm E House 6 Flock 2016-11', 24: 'Farm E House 8 Flock 2016-11', 25: 'Farm E House 7 Flock 2017-01', 26: 'Farm E House 8 Flock 2017-01', 27: 'Farm E House 8 Flock 2017-04', 28: 'Farm E House 7 Flock 2017-04

In [3]:
def reduceDot(linex, liney):
    new_list = []
    for i in range(len(linex)):
        new_list.append(linex[i] or liney[i])
    return new_list

checkAllDot = cleanData.map(lambda x: [('.' in i) for i in x])\
                       .reduce(lambda x, y: reduceDot(x, y))
print(checkAllDot)

def numberCleaner(line):
    """
    This function needs to be improved
    """
    resultList = []
    item = None
    for idx in range(len(line)):
        if idx <= 12 or idx == 38:
            if idx == 6: 
                if not line[idx]: item = -1
                else:
                    item = line[idx].strip().replace(',', '').replace(' ', '')
                    item = int(item)
            else: 
                if not line[idx]: item = ""
                else: item = line[idx]
        elif checkAllDot[idx]:
            if not line[idx]: item = -0.1
            else:
                item = line[idx].strip().replace(',', '').replace(' ', '')
                item = float(item)
        else: 
            if not line[idx]: item = -1
            else:
                item = line[idx].strip().replace(',', '').replace(' ', '').replace('"', '')
                item = int(item)
        resultList.append(item)
    return resultList

numCleanData = cleanData.map(numberCleaner)

def transferNone(line):
    new_line = []
    for item in line:
        if type(item) != str:
            if item < 0: new_line.append(None)
            else: new_line.append(item)
        else:
            if not item: new_line.append(None)
            else:new_line.append(item)
    return new_line

broilers = numCleanData.map(transferNone)

broilersDF = sqlContext.createDataFrame(numCleanData, header)

[False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, True, False, False, False, False, False, True, True, True, True, True, True, True, True, True, True, True, True, True, True]


In [4]:
from pyspark.sql.types import BooleanType
from pyspark.sql.functions import udf

def selectArea(Main_Keys, inCols):
    selectRows = udf(lambda x: x in Main_Keys, BooleanType())
    area = broilersDF.filter(selectRows(broilersDF.Main_Key)).select(inCols)
    return area

print(header)
print(mainKeys)

test = ['Age', 'Death']
samples = [[] for i in range(len(mainKeys))]
for i in range(len(mainKeys)):
    samples[i] = selectArea(mainKeys[i], test)

sample_train = selectArea(mainKeys[2:], test)
sample_test = selectArea(mainKeys[1:3], test)

print(sample_train.count())
print(sample_test.count())

['Main_Key', 'Date', 'Customer_Code', 'House', 'Flock', 'Gene_Line', 'Birds_Begin', 'Hatch_Date', 'Arrive_Date', 'Remove_date', 'Deactivate_Date', 'Veterinarian', 'Hatchery', 'Age', 'Birds_Present', 'Birds_thinned', 'Death', 'Death_Cum', 'Total_Death_Rate', 'Alive_Rate', 'Body_Weight_g', 'Uniformity_Rate', 'Daily_Gaing', 'Avg_Daily_Gaing_Per_Day', 'Feed_Intake_Per_house_kg', 'FCR_Cum', 'Wheat_Per_Bird_Cum', 'Wheat_Per_Bird', 'Wheat_Per_Day', 'Feed_Intake_Per_Bird_Housed_Cum_kg', 'Feed_Intake_Per_Bird_g', 'Wheat_g', 'FCR', 'Water_l', 'Water_Intake_Per_Bird_ml', 'Water_Intake_Per_Bird_Cum_l', 'Water_Feed', 'Water_FeedCum', 'Comment']
['Farm A House 1 Flock 2016-07', 'Farm A House 1 Flock 2016-10', 'Farm B House 3 Flock 2016-11', 'Farm B House 1 Flock 2016-11', 'Farm B House 3 Flock 2016-12', 'Farm B House 2 Flock 2017-01', 'Farm B House 2 Flock 17-3', 'Farm B House 1 Flock 17-3', 'Farm C House 2 Flock Koppel 11 stal 2', 'Farm D House 1 Flock 2016-07', 'Farm D House 3 Flock 2016-07', 'Far

In [ ]:
# This cell is for preparing datasets used in regression




In [7]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.regression import DecisionTreeRegressor, LinearRegression, LinearRegressionModel, RandomForestRegressor

def Do_Machine_Learning(trainingSet, testSet, xCols, yValues, regressor = LinearRegression(),
                         paramGrid = [], evalMetric = "rmse", seed = None):
    """
    Return (<object> model, <float> error_estimate, <dataframe> result)
    
    trainingSet: dataframe, used to train model
    testSet: dataframe, used to feed the model and get the result(and error estimate)
    xCols: list of strings, names of columns which used as inputs
    yValues: string, name of column that contains dependent values 
    regressor: Regression object, by default = LinearRegression()
    paramGrid: list built byParamGridBuilder, by default = empty list
    evalMetric: string, name of matrix used for evaluation, by default = "rmse"
    seed: int or None, seed for random number generator, if == None will use random numbers
    
    !!! seed is useless at that time !!!
    """
    # push estimator into pipeline
    vec = VectorAssembler(inputCols = xCols, outputCol = "features")
    regPipeline = Pipeline()
    regPipeline.setStages([vec, regressor])   
    # build evaluator
    regEval = RegressionEvaluator(predictionCol = "Predicted", labelCol = yValues, metricName = evalMetric)
    # combine estimator and evaluator to a cross validator
    crossval = CrossValidator(estimator = regPipeline, evaluator = regEval, numFolds = 3)
    # set parameters grid
    crossval.setEstimatorParamMaps(paramGrid)
    # trainning
    regModel = crossval.fit(trainingSet).bestModel
    # predicting
    predictions = regModel.transform(testSet)
    # get evaluating result
    evaluation = regEval.evaluate(predictions)
    
    return regModel, evaluation, predictions

# build regressor
lr1 = LinearRegression()
lr1.setPredictionCol("Predicted")\
   .setLabelCol("Death")

# build parameter grid
regParam = [x / 100.0 for x in range(1, 10)]
pg1 = (ParamGridBuilder()
             .addGrid(lr1.regParam, regParam)
             .build())

print(type(lr1))
print(type(pg1))

# run ML
model, result, predictionDF = Do_Machine_Learning(sample_train, sample_test, ["Age"], "Death", lr1, pg1)

# print attributions of model
"""
print("attributes of the model are: {}".format(dir(model)))
print("method list: {}".format([method for method in dir(model) if callable(getattr(model, method))]))
print(model.stages)
"""

# Print coefficients and intercept
weights = model.stages[1].coefficients
print(list(zip(["Age"], weights)))
print(model.stages[1].intercept)

# print error and result
print("New Root Mean Squared Error: {0:2.2f}\n".format(result))
print(predictionDF.show())

# print the model
print(model.stages)

attributes of the model are: ['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__metaclass__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_clear', '_copyValues', '_copy_params', '_defaultParamMap', '_dummy', '_from_java', '_paramMap', '_params', '_randomUID', '_resetUid', '_resolveParam', '_set', '_setDefault', '_shouldOwn', '_to_java', '_transform', 'copy', 'explainParam', 'explainParams', 'extractParamMap', 'getOrDefault', 'getParam', 'hasDefault', 'hasParam', 'isDefined', 'isSet', 'load', 'params', 'read', 'save', 'stages', 'transform', 'uid', 'write']
method list: ['__class__', '__delattr__', '__dir__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '_

In [9]:
print(type(lr1))
print(pg1)

<class 'pyspark.ml.regression.LinearRegression'>
[{Param(parent='LinearRegression_4dc5b27433f5df6b3c15', name='regParam', doc='regularization parameter (>= 0).'): 0.01}, {Param(parent='LinearRegression_4dc5b27433f5df6b3c15', name='regParam', doc='regularization parameter (>= 0).'): 0.02}, {Param(parent='LinearRegression_4dc5b27433f5df6b3c15', name='regParam', doc='regularization parameter (>= 0).'): 0.03}, {Param(parent='LinearRegression_4dc5b27433f5df6b3c15', name='regParam', doc='regularization parameter (>= 0).'): 0.04}, {Param(parent='LinearRegression_4dc5b27433f5df6b3c15', name='regParam', doc='regularization parameter (>= 0).'): 0.05}, {Param(parent='LinearRegression_4dc5b27433f5df6b3c15', name='regParam', doc='regularization parameter (>= 0).'): 0.06}, {Param(parent='LinearRegression_4dc5b27433f5df6b3c15', name='regParam', doc='regularization parameter (>= 0).'): 0.07}, {Param(parent='LinearRegression_4dc5b27433f5df6b3c15', name='regParam', doc='regularization parameter (>= 0).'